# Biological introduction

## Interhepatic Cholangiocarcinoma
Intrahepatic Cholangiocarcinoma (ICC) is a type of malignant solid tumour arising from the [bile ducts in the liver](https://doi.org/10.2147/OTT.S93629). Clinical management of the disease is difficult, with only a minority of cases available for surgical treatment (tumor resection, ~15%), most studies reporting systemic chemotherapy to be ineffective in extending lifespan or improving other clinical outcomes (TODO find sources), and with reports showing moderately successful clinical outcomes of hepatic arterial infusion chemotherapy using generic chemotherapeutics such as floxuridine or yttrium-90 (TODO find sources). The primary clinical outcome that can be achieved with such localized chemotherapy is downsizing of the tumor, which can allow for successful re-evaluation of patient's eligibility for resection. Targeted therapy was shown to be [effective in isolated cases](https://doi.org/10.3978/j.issn.2078-6891.2013.031), but devising eligibility criteria for existing targeted therapies remains an open challenge (TODO link some papers on molecular biomarkers for tageted therapies, or maybe Ruibin's paper if it's already available). Given limited efficacy of existing treatment options, and poor prognosis, with median survival of only about 6 months, there's a pressing need to develop new treatments. This proves to be difficult as ICC is characterised by a high level of molecular diversity, and has a number of clinical presentations. Due to recent experimental advances it has become possible to study molecular diversity of thousands of individual cancer cells, using an approach called droplet-based single cell RNA-seq (scRNA-seq).

## Single cell RNA-seq

A single cell droplet-based RNA-seq system uses a microfluidics device, thousands of barcoding beads, and a DNA sequencer to inspect molecular identity of individual cells. The microfluidics device allows for the formation of nanolitre droplets, some of which capture individual cancer cells, some of which capture individual barcoding beads, and some of which capture both a cell and a bead. This final group of droplets containing a cell and a bead participate in further analysis, which consists of cell lysis (breaking cells apart), RNA capture by the barcoding bead, reverse transcription into DNA an DNA amplification. The resulting collection of DNA molecules directly corresponds to the original RNA molecules captured by the beads, with each DNA molecule directly traceable to a single cell, through a cell-specific barcode (CB), and even to a single RNA molecule, through unique molecular identifier (UMI). All DNA molecules are then combined together, and sequenced using a next-generation sequencer (NGS). The resulting dataset, after several steps of pre-processing can be represented as a matrix, with the number of columns equal to the number of (human) genes, the number of rows equal to the number of detected cells, and integer values representing number of the individual RNA molecules expressed per cell per gene. This, in effect, represents a state of the art measurment of the molecular identity of the cell -- and in the context of ICC, will allow us to learn more about development of this cancer, confirm existing understanding of the molecular basis of the disease, and hopefully bring new insights, such as candidate treatment targets.

# Computational introduction

The analysis is carried out in [scanpy](https://github.com/theislab/scanpy), a fantastic open-source single-cell Python package.

There is a number of supportive packages we have to import, most importantly `anndata`, a matrix-based data structure, which can be used to store transcript counts per cell per gene.

Anndata supports multiple layers of the count matrix, which can be used to store counts of both mature, spliced mRNA molecules, and nascent, unspliced ones. Layers make it possible to use scanpy in analysing not only the position of individual cells, but also the direction of their development, dubbed "RNA velocity". This is achieved with the `scvelo` package.

In [1]:
import numpy as np
import pandas
import scanpy as sc
import anndata
import json
import matplotlib.pyplot as plt
import scvelo as scv
import os
import requests
import shutil
%matplotlib inline

/home/picrin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Downloading and loading the loom file

We've already carried out alignement, barcode correction, some initial filtering, and spliced and unspliced transcript counting, using respectively the `STAR` aligner, `dropest` barcode correction and `velocyto` to produce the loom file (we do not use its computed velocity field). We can download the loom file and load it into an `anndata` object using scanpy.

In [2]:
bucket = "https://s3.eu-west-2.amazonaws.com/public-bile-duct-cancer/"
filename = "bile_duct_cancer_single_cell.loom"

In [3]:
# We only download the file if it isn't present locally 
if filename not in os.listdir("data"):
    print("downloading loom file")
    url = bucket + filename
    r = requests.get(url, verify=True, stream=True)
    with open(os.path.join("data", filename), 'wb') as f:
        shutil.copyfileobj(r.raw, f)

In [4]:
adata = sc.read_loom(filename)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


It's a good idea to poke around a little bit and check out our data before we analyse it.

In [5]:
adata

AnnData object with n_obs × n_vars = 2104 × 58395 
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced'

Let's note that `n_obs` is the number of individual cancer cells -- after barcode correction and initial quality filtering. `n_vars` is the number of detected genes. Both dimensions will be reduced by further quality checks.

Let's check out the layers

In [6]:
adata.layers.as_dict()

{'ambiguous': <2104x58395 sparse matrix of type '<class 'numpy.float32'>'
 	with 639413 stored elements in Compressed Sparse Row format>,
 'matrix': <2104x58395 sparse matrix of type '<class 'numpy.float32'>'
 	with 4690195 stored elements in Compressed Sparse Row format>,
 'spliced': <2104x58395 sparse matrix of type '<class 'numpy.float32'>'
 	with 3676753 stored elements in Compressed Sparse Row format>,
 'unspliced': <2104x58395 sparse matrix of type '<class 'numpy.float32'>'
 	with 1076388 stored elements in Compressed Sparse Row format>}

As we can see, there are multiple matrices stored in the `anndata` object. `spliced` and `unspliced` are the ones we'll be interested in for the velocity computation.

Layers are only used for RNA velocity. Other types of analysis are carried out using just a single matrix, stored as a field in the `anndata` object, e.g. `adata.X`

In [7]:
type(adata.X)

scipy.sparse.csr.csr_matrix

As we can see, this is identical to the "spliced" matrix from the layers. This makes sense as mature (spliced) messenger RNA (mRNA) is what we're most interested in, and should abundant, since most experimental techniques (including our technique, Drop-seq) enrich for the polyA cap, characteristic of mature mRNA, and using a polyT cDNA sequence present at the end of the bead used to capture RNA.

In [8]:
np.allclose(adata.X.toarray(), adata.layers.as_dict()["spliced"].toarray())

True

We can inspect the column (gene) information. Accession is a unique, immutable gene identifier, which tends to be more machine-friendly than still evolving gene nomenclature (e.g. CELF1 is sometimes referred to as CUGBP1). Chromosome, End, Start and Strand uniquely identify the position of the gene in a reference human chromosome (we use GRCh38).

In [9]:
adata.var[:10]

,Accession,Chromosome,End,Start,Strand
WASH7P,ENSG00000227232,1,29570,14404,-
MIR6859-1,ENSG00000278267,1,17436,17369,-
FAM138A,ENSG00000237613,1,36081,34554,-
AL627309.1,ENSG00000238009,1,133723,89295,-
AL627309.3,ENSG00000239945,1,91105,89551,-
AL627309.6,ENSG00000268903,1,135895,135141,-
AL627309.7,ENSG00000269981,1,137965,137682,-
AL627309.2,ENSG00000239906,1,140339,139790,-
AL627309.5,ENSG00000241860,1,173862,141474,-
RNU6-1100P,ENSG00000222623,1,157887,157784,-


In [10]:
"CUGBP1" in adata.var_names

False

In [11]:
"CELF1" in adata.var_names

True

row (cell) information is less interesting, since at the beginning of the analysis we know very little about any particular cell -- the only information we have is the cellular barcode. In fact, the whole analysis can be viewed as transferring existing information we have about genes to information about cells, all done via the count matrices.

In [12]:
adata.obs[:10]

""
correct_result_9QWIF:AATCCTACATATx
correct_result_9QWIF:AAATGTAAGACTx
correct_result_9QWIF:AAGCAGGTGCTTx
correct_result_9QWIF:AAACACCAGCATx
correct_result_9QWIF:AAGCTCATGCGGx
correct_result_9QWIF:AAGTGATTGGAGx
correct_result_9QWIF:AATCGCTGTTTTx
correct_result_9QWIF:AACGGGTTCGTGx
correct_result_9QWIF:AACCTCTCTTCTx
correct_result_9QWIF:AAGCAAGTAACTx


# Write to file

We can save the dataset locally, using a scanpy-native file format, to make it easily available for downstream analysis.

In [13]:
adata.write(os.path.join("data", "00_bile_duct_cancer.h5ad"))

... storing 'Chromosome' as categorical
... storing 'Strand' as categorical


# Exercises

1. Using numpy, compute a total number of genes per cell, and save it in the `obs` variable (Hints: read about `np.ravel` and `np.sum`, you'll have to work with `adata.X`).
2. Plot your results from the exercise 1 using either a histogram or a violin plot.

You can find solutions to the exercises in the next notebook, `01_filtering_and_normalisation.ipynb`

# What to read next

The next notebook, `01_filtering_and_normalisation.ipynb` is quite technical, unless you find normalisation particularly interesting, I'd suggest to skip it and go straight to `02_PCA_plots_and_stemness_analysis.ipynb`